In [162]:
#from google.colab import drive
#drive.mount('/content/drive/')

In [163]:
import numpy as np
import pandas as pd
import ast
import warnings
import joblib
warnings.filterwarnings("ignore")

In [164]:
dataset = pd.read_csv('RegressionTestSet.csv')
dataset.head()

,date,airline,ch_code,num_code,dep_time,time_taken,stop,arr_time,type,route,price
0,8/3/2022,Air India,AI,430,9:55,11h 20m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,21:15,economy,"{'source': 'Chennai', 'destination': 'Mumbai'}","4,357"
1,7/3/2022,Indigo,6E,926,18:30,04h 55m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:25,economy,"{'source': 'Delhi', 'destination': 'Mumbai'}","4,270"
2,24-03-2022,Indigo,6E,6491,13:05,07h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:45,economy,"{'source': 'Bangalore', 'destination': 'Mumbai'}","3,153"
3,27-03-2022,Air India,AI,473,18:40,22h 25m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,17:05,economy,"{'source': 'Delhi', 'destination': 'Bangalore'}","4,728"
4,7/3/2022,Indigo,6E,684,21:55,01h 15m,non-stop,23:10,economy,"{'source': 'Bangalore', 'destination': 'Hydera...","1,714"


In [165]:
dataset['full_information_timestamp'] = 0
dataset['flight_day'] = 0
dataset['flight_month'] = 0
dataset['week_day_of_flight'] = 0
dataset['departure_time_of_the_day']=0
dataset['arrival_time_of_the_day']=0
dataset['num_of_hours_taken'] =0
dataset['num_of_stops'] = 0
dataset['source'] = 0
dataset['destination'] = 0
dataset['distance_between_2_cities'] = 0
dataset['airline_Air India']=0
dataset['airline_Other_airline'] = 0
dataset['airline_Vistara']=0
dataset['airline_Indigo']=0
dataset['source_Bangalore']=0
dataset['source_Delhi']=0
dataset['source_Hyderabad']=0
dataset['source_Kolkata']=0
dataset['source_Chennai']=0
dataset['source_Mumbai']=0
dataset['destination_Bangalore']=0
dataset['destination_Chennai']=0
dataset['destination_Delhi']=0
dataset['destination_Kolkata']=0
dataset['destination_Mumbai']=0
dataset['destination_Hyderabad']=0
dataset['week_day_of_flight_Saturday']=0
dataset['week_day_of_flight_Sunday']=0
dataset['week_day_of_flight_Tuesday']=0
dataset['week_day_of_flight_Thursday']=0
dataset['dep_time_Afternoon']=0
dataset['dep_time_Early morning']=0
dataset['dep_time_Evening']=0
dataset['dep_time_Morning']=0
dataset['arr_time_Afternoon']=0
dataset['arr_time_Early morning']=0
dataset['arr_time_Evening']=0
dataset['arr_time_Morning']=0
dataset['formatted_price'] =0

In [166]:
def format_date(date):
    dashed_date = date.replace('/','-')
    return dashed_date

In [167]:
dataset['full_information_timestamp'] = dataset.date.apply(lambda date:format_date(date))

In [168]:
def extract_day_month(date):
  splitted_date = date.split('-')
  day = int(splitted_date[0])
  month =int(splitted_date[1])
  if splitted_date[1]== '2' or splitted_date[1]== '02':
      new_date_format = 'February ' + splitted_date[0] + ', '+ splitted_date[2]
  else :
      new_date_format = 'March ' + splitted_date[0] + ', '+ splitted_date[2]
  return day,month,new_date_format

In [169]:
dataset[['flight_day', 'flight_month','full_information_timestamp']] = pd.DataFrame(dataset.full_information_timestamp.apply(lambda date: extract_day_month(date)).tolist(), index=dataset.index)

In [170]:
dataset['full_information_timestamp'] = pd.to_datetime(dataset['full_information_timestamp'])

In [171]:
def extract_weekday(day):
    return day.day_name()

In [172]:
dataset['week_day_of_flight'] = dataset.full_information_timestamp.apply(lambda date:extract_weekday(date))

In [173]:
def remove_low_frequency_airlines():
    high_frequency_airlines =['Vistara','Air India','Indigo']
    dataset ['airline'] = [airline if airline in high_frequency_airlines else 'Other_airline' for airline in dataset['airline']]
   

In [174]:
remove_low_frequency_airlines()

In [175]:
def calculate_time_taken(time_taken):
      time = time_taken.split(' ')
      float_hour = time[0].split('.')
      hours = 0
      if(len(float_hour)>1):
          time[0] = float_hour[0] + 'h'
          time[1]= float_hour[1][:-1] + time[1]
      if time[1][:-1] != '' : 
          if int(time[1][:-1]) >= 40 :
              hours = int(time[0][:-1]) + 1
          else :
              hours = int(time[0][:-1]) 
      else :
          hours = int(time[0][:-1])
      return hours

In [176]:
dataset['num_of_hours_taken'] = dataset.time_taken.apply(lambda time_taken: calculate_time_taken(time_taken))

In [177]:
def categorize_time(time):
    hours = time.split(':')
    hours[0] = int(hours[0])
    if hours[0] <6:
        return 'Early morning'
    elif 6<= hours[0] < 12 :
        return 'Morning'
    elif 12<= hours[0] < 18:
        return 'Afternoon'
    else:
        return 'Evening'

In [178]:
dataset['dep_time'] = dataset.dep_time.apply(lambda time: categorize_time(time))

In [179]:
dataset['arr_time'] = dataset.arr_time.apply(lambda time: categorize_time(time))

In [180]:
def split_num_of_stops(stop):
    if stop[:8] == 'non-stop':
        stop_count = 0
    elif stop[:6] == '1-stop':
        stop_count = 1
    else:
        stop_count = 2
    return stop_count

In [181]:
dataset['num_of_stops']= dataset.stop.apply(lambda stop:split_num_of_stops(stop))

In [182]:
dataset['type'] = [1 if kind_of_trip == 'business' else 0 for kind_of_trip in dataset['type']]

In [183]:
def split_route(route):
    path = ast.literal_eval(route)
    source = path['source']
    destination = path['destination']
    return source,destination

In [184]:
dataset[['source', 'destination']] = pd.DataFrame(dataset.route.apply(lambda route: split_route(route)).tolist(), index=dataset.index)

In [185]:
distances = dict()
distances['Mumbai','Kolkata'] = 2167
distances['Mumbai','Hyderabad'] = 721
distances['Mumbai','Chennai'] = 1344
distances['Mumbai','Bangalore'] = 995
distances['Mumbai','Delhi'] = 1439
distances['Delhi','Bangalore'] = 2169
distances['Delhi','Kolkata'] = 1554
distances['Delhi','Hyderabad'] = 1579
distances['Delhi','Chennai'] = 2202
distances['Bangalore','Kolkata']= 1560
distances['Bangalore','Hyderabad'] = 569
distances['Bangalore','Chennai'] = 348
distances['Kolkata','Hyderabad'] = 1489
distances['Kolkata','Chennai'] = 1663
distances['Hyderabad','Chennai'] = 628

In [186]:
def calculate_distance(source,destination):
    try :
        dist = distances[source,destination]
    except :
        dist = distances[destination,source]
    return dist

In [187]:
dataset['distance_between_2_cities'] = dataset.apply(lambda dataFrame: calculate_distance(dataFrame['source'], dataFrame['destination']), axis=1)
dataset['distance_between_2_cities'] = dataset['distance_between_2_cities']/2202

In [188]:
def fix_price_format(price):
    number = price.split(',')
    cost = number[0] + number[1]
    cost=int(cost)
    return cost

In [189]:
dataset['formatted_price'] = dataset.price.apply(lambda price:fix_price_format(price))

In [190]:
dataset.drop(columns=['date','ch_code','time_taken','stop','route','price','full_information_timestamp','num_code','flight_day'],axis = 1,inplace=True)

In [191]:
def one_hot_encode_time(time):
    morning =0
    afternoon =0
    evening = 0
    early_morning = 0
    if time =='Early morning':
        early_morning = 1
    elif time == 'Afternoon':
        afternoon =1
    elif time == 'Morning':
        morning = 1
    elif time == 'Evening' :
        evening = 1
    return early_morning,morning,afternoon,evening

In [192]:
dataset[['dep_time_Early morning', 'dep_time_Morning','dep_time_Afternoon','dep_time_Evening']] = pd.DataFrame(dataset.dep_time.apply(lambda time: one_hot_encode_time(time)).tolist(), index=dataset.index)

In [193]:
dataset[['arr_time_Early morning', 'arr_time_Morning','arr_time_Afternoon','arr_time_Evening']] = pd.DataFrame(dataset.arr_time.apply(lambda time: one_hot_encode_time(time)).tolist(), index=dataset.index)

In [194]:
def one_hot_encode_week_day(day):
    sunday = 0
    saturday = 0
    thursday = 0
    tuesday = 0
    if day =='Tuesday' :
        tuesday = 1
    elif day =='Thursday' :
        thursday = 1
    elif day =='Saturday' :
        saturday = 1
    elif day == 'Sunday' :
        sunday = 1
    return saturday,sunday,tuesday,thursday

In [195]:
dataset[['week_day_of_flight_Saturday', 'week_day_of_flight_Sunday','week_day_of_flight_Tuesday','week_day_of_flight_Thursday']] = pd.DataFrame(dataset.week_day_of_flight.apply(lambda day: one_hot_encode_week_day(day)).tolist(), index=dataset.index)

In [196]:
def one_hot_encode_airline(airline):
    vistara = 0
    indigo = 0
    air_india = 0
    other = 0
    if airline == 'Vistara' :
        vistara = 1
    elif airline == 'Indigo' :
        indigo = 1
    elif airline == 'Air India' :
        air_india = 1
    elif airline =='Other_airline' :
        other = 1
    return vistara,air_india,indigo,other

In [197]:
dataset[['airline_Vistara', 'airline_Air India','airline_Indigo','airline_Other_airline']] = pd.DataFrame(dataset.airline.apply(lambda airline: one_hot_encode_airline(airline)).tolist(), index=dataset.index)


In [198]:
def one_hot_encode_city(city):
    delhi = 0
    hyderabad = 0
    mumbai = 0
    bangalore = 0
    kolkata = 0
    chennai =0
    if city == 'Delhi' :
        delhi = 1
    elif city == 'Hyderabad' :
        hyderabad =1
    elif city == 'Mumbai' :
        mumbai = 1
    elif city == 'Bangalore' :
        bangalore = 1
    elif city == 'Kolkata' :
        kolkata = 1
    elif city == 'Chennai' :
        chennai = 1
    return delhi,hyderabad,mumbai,bangalore,kolkata,chennai

In [199]:
dataset[['source_Delhi','source_Hyderabad', 'source_Mumbai','source_Bangalore','source_Kolkata','source_Chennai']] = pd.DataFrame(dataset.source.apply(lambda city: one_hot_encode_city(city)).tolist(), index=dataset.index)

In [200]:
dataset[['destination_Delhi','destination_Hyderabad', 'destination_Mumbai','destination_Bangalore','destination_Kolkata','destination_Chennai']] = pd.DataFrame(dataset.destination.apply(lambda city: one_hot_encode_city(city)).tolist(), index=dataset.index)


In [201]:
dataset

,airline,dep_time,arr_time,type,flight_month,week_day_of_flight,departure_time_of_the_day,arrival_time_of_the_day,num_of_hours_taken,num_of_stops,...,week_day_of_flight_Thursday,dep_time_Afternoon,dep_time_Early morning,dep_time_Evening,dep_time_Morning,arr_time_Afternoon,arr_time_Early morning,arr_time_Evening,arr_time_Morning,formatted_price
0,Air India,Morning,Evening,0,3,Tuesday,0,0,11,1,...,0,0,0,0,1,0,0,1,0,4357
1,Indigo,Evening,Evening,0,3,Monday,0,0,5,1,...,0,0,0,1,0,0,0,1,0,4270
2,Indigo,Afternoon,Evening,0,3,Thursday,0,0,8,1,...,1,1,0,0,0,0,0,1,0,3153
3,Air India,Evening,Afternoon,0,3,Sunday,0,0,22,1,...,0,0,0,1,0,1,0,0,0,4728
4,Indigo,Evening,Evening,0,3,Monday,0,0,1,0,...,0,0,0,1,0,0,0,1,0,1714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,Air India,Morning,Morning,0,2,Tuesday,0,0,26,1,...,0,0,0,0,1,0,0,0,1,11409
19996,Air India,Morning,Morning,0,3,Saturday,0,0,22,1,...,0,0,0,0,1,0,0,0,1,5915
19997,Vistara,Morning,Evening,0,2,Monday,0,0,15,1,...,0,0,0,0,1,0,0,1,0,11843
19998,Air India,Afternoon,Morning,1,3,Thursday,0,0,17,1,...,1,1,0,0,0,0,0,0,1,47545


In [202]:
features = ['type', 'flight_month', 'num_of_stops',
       'distance_between_2_cities', 'formatted_price', 'airline_Air India',
       'airline_Indigo', 'airline_Other_airline', 'airline_Vistara',
       'source_Bangalore', 'source_Chennai', 'source_Delhi',
       'source_Hyderabad', 'source_Kolkata', 'source_Mumbai',
       'destination_Bangalore', 'destination_Chennai', 'destination_Delhi',
       'destination_Hyderabad', 'destination_Kolkata', 'destination_Mumbai',
       'week_day_of_flight_Saturday', 'week_day_of_flight_Sunday',
       'week_day_of_flight_Thursday', 'week_day_of_flight_Tuesday',
       'dep_time_Afternoon', 'dep_time_Early morning', 'dep_time_Evening',
       'dep_time_Morning', 'arr_time_Afternoon', 'arr_time_Early morning',
       'arr_time_Evening', 'arr_time_Morning']

In [203]:
for column in dataset.columns:
    if column not in features:
        dataset.drop(columns=column,axis=1,inplace=True)

In [204]:
dataset = dataset[['type', 'flight_month', 'num_of_stops',
       'distance_between_2_cities', 'formatted_price', 'airline_Air India',
       'airline_Indigo', 'airline_Other_airline', 'airline_Vistara',
       'source_Bangalore', 'source_Chennai', 'source_Delhi',
       'source_Hyderabad', 'source_Kolkata', 'source_Mumbai',
       'destination_Bangalore', 'destination_Chennai', 'destination_Delhi',
       'destination_Hyderabad', 'destination_Kolkata', 'destination_Mumbai',
       'week_day_of_flight_Saturday', 'week_day_of_flight_Sunday',
       'week_day_of_flight_Thursday', 'week_day_of_flight_Tuesday',
       'dep_time_Afternoon', 'dep_time_Early morning', 'dep_time_Evening',
       'dep_time_Morning', 'arr_time_Afternoon', 'arr_time_Early morning',
       'arr_time_Evening', 'arr_time_Morning']]

In [205]:
label = dataset['formatted_price']
dataset.drop(columns=['formatted_price'],axis=1,inplace=True)
features = dataset.copy()

In [206]:
LinearRegression = joblib.load('LR.h5')
LinearRegression.score(features,label)

0.9078899061500766

In [207]:
PoissonRegressor = joblib.load('PR.h5')
PoissonRegressor.score(features,label)

0.9326985983866928

In [208]:
BaggingRegressor = joblib.load('BR.h5')
BaggingRegressor.score(features,label)

0.9550986116988044

In [209]:
ExtraTreeRegressor = joblib.load('ETR.h5')
ExtraTreeRegressor.score(features,label)

0.9543657176196503

In [210]:
RandomForest = joblib.load('RF.h5')
RandomForest.score(features,label)

0.9555539122508884

In [211]:
DecisionTree = joblib.load('DT.h5')
DecisionTree.score(features,label)

0.954300712746952

In [212]:
LightGradientBoostingMachine = joblib.load('LGBMR.h5')
LightGradientBoostingMachine.score(features,label)

0.9552752246038918

In [213]:
HistGradientBoosting = joblib.load('HGBR.h5')
HistGradientBoosting.score(features,label)

0.9552455633445458